# Train T5 with SFT
Steps to run:
* Add the following to secret keys: Github PAT, HF Token (write access)


In [2]:
import os
import sys
from google.colab import userdata

# ADD A PAT https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')  # put the token in colab secret keys
os.environ['REPOSITORY'] =  "github.com/jth500/maet-pln.git"
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
!git clone -b refactor https://${{GITHUB_TOKEN}}@${{REPOSITORY}}
!mv maet-pln maet_pln # rename to remove the invalid dash
sys.path.append("maet_pln/src")

Cloning into 'maet-pln'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 547 (delta 67), reused 79 (delta 41), pack-reused 416
Receiving objects: 100% (547/547), 624.41 KiB | 5.53 MiB/s, done.
Resolving deltas: 100% (340/340), done.


In [3]:
!pip install -q pyarrow==12.0.0
# !pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q transformers
!pip install -q tdqm torch
!pip install -q -U bitsandbytes
!pip install -q -U evaluate
# !pip install -q evaluate rouge_score
!pip install python-dotenv
# !pip install peft
# !pip install cohere
# !pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.

In [4]:
import random
import torch
from datasets import (load_dataset,
                      Dataset)
from tqdm import tqdm
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          Trainer,
                          GenerationConfig,
                          pipeline)
import evaluate


from datasets import get_dataset_split_names
import json
import pandas as pd

In [6]:
from data_handler.data_handler import T5DatasetHandler

In [7]:
DATASET_NAME = "EdinburghNLP/xsum"
dh = T5DatasetHandler(DATASET_NAME, tokenizer=None, data_size=100, push_to_hub=True, save_locally=False, rlaif=False)
data = dh.process_data()
tokenizer = dh.tokenizer

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Generating train split: 0 examples [00:00, ? examples/s]

...

Filter:   0%|          | 0/90 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


In [8]:
sft_train_data, val_data = data
print([len(dataset) for dataset in data])

[90, 10]


In [9]:
### SFT
from sft import SFT
from model_builder import T5ModelBuilder
import torch
torch.device("cuda")



device(type='cuda')

In [10]:
# SET UP BASE MODEL
save_dir = "sft-t5-xsum-0504"
base_model_id = "t5-base"
model_builder = T5ModelBuilder(base_model_id, tokenizer=tokenizer)
base_model = model_builder.base_model
stf_trainer = SFT(
    save_dir=save_dir,
    tokenizer=tokenizer,
    base_model=model_builder.base_model,
    train_dataset=sft_train_data
)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
torch.cuda.empty_cache()
stf_trainer.train_model()
stf_trainer.push_model_to_hub()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1834: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case jth500/sft-t5-xsum-0504).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
36,6.600500
72,5.015100
108,4.306400
144,4.059400
180,3.963700


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

# Inference

In [12]:

from transformers import pipeline
summarizer = pipeline('summarization', model="jth500/" + save_dir, tokenizer=tokenizer)

In [14]:
prompts = list(val_data['input'])
prompts[1]

'summarize: Big oil producing nations meet this weekend in Qatar to discuss plans for a freeze in production levels.\nBut there is scepticism over whether such a deal would make much difference to the current oversupply of oil in the market.\nThe Dow Jones Industrial Average fell 28.97 points to 17,897.46.\nThe S&P 500 was down 2.05 points to 2,080.73. The technology focused Nasdaq Composite slipped 7.67 points to 4,938.22.\nStocks in the energy sectors fell as another US bank - Citigroup - reported it was setting cash aside to cover losses on loans to the energy industry.\nShares in banking giant Citigroup closed 0.13% down after trading mostly higher all day. It reported a 27% fall in quarterly profit, but that was not as bad as analysts had been expecting.\nShares of Marathon Petroleum were down 3.8%, Exxon Mobile fell 0.5% and Chevron was 0.75% lower.\nShares of Apple dropped 2% after a report suggested the company was cutting iPhone production in the April-June quarter due to weak

In [15]:
responses = []
summaries = []
indices = []

for i, p in enumerate(tqdm(prompts)):
  try:
    r = summarizer(p, max_length=10000, truncation=True)
    responses.append(r[0]['summary_text'])
    summaries.append(val_data[i]['output'])
    indices.append(i)
  except:
    pass

  0%|          | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 10000, but your input_length is only 884. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=442)
100%|██████████| 10/10 [02:28<00:00, 14.80s/it]


In [16]:
df = pd.DataFrame({'summary_pred': responses, 'summary_actual': summaries, 'index': indices,
                   'input': val_data[indices]['input']})
df.head()

,summary_pred,summary_actual,index,input
0,pte Paul Wilkinson said he asked Pte Cheryl Ja...,<s>A boyfriend of a young soldier who died at ...,0,summarize: Pte Paul Wilkinson said he asked Pt...
1,big oil producing nations meet this weekend in...,<s>(Close):Wall Street markets fell on Friday ...,1,summarize: Big oil producing nations meet this...
2,the 27-year-old midfielder missed the 2014 wor...,<s>Borussia Dortmund's Marco Reus has been omi...,2,"summarize: Reus, 27, missed the 2014 World Cup..."
3,"Jardim says he feels part of the ""principality...",<s>Monaco boss Leonardo Jardim has been reward...,3,"summarize: Jardim, in charge since 2014, descr..."
4,the commons is debating the welfare benefit up...,<s>The government has urged MPs to back a 1% c...,4,summarize: Work and Pensions Secretary Iain Du...
